In [ ]:
%matplotlib inline
import sklearn.cluster as cluster
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
import math
from sklearn.neighbors import KNeighborsClassifier as KNNCLF
from sklearn.neighbors import KNeighborsRegressor as KNNREG
from sklearn.preprocessing import Normalizer as normalizer
from sklearn.cross_validation import cross_val_score as cross_val
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
#from sklearn.neural_network import MLPClassifier
from scipy import stats
from sklearn.feature_selection import RFE

In [ ]:
Crimes = pd.read_csv('crime.csv')
Crimes = Crimes[np.isfinite(Crimes['Police_Districts'])]
Crimes = Crimes[np.isfinite(Crimes['UCR_General'])]
CrimesData = Crimes.as_matrix()
datainit = CrimesData[:,[10,2,5,8,12,13]]

In [ ]:
data = datainit

In [ ]:
newData = []
for item in data:
    date = ''.join( [c for c in item[1] if c not in ' -:' ] )
    year = date[:4]
    month = date[4:6]
    day = date[6:8]
    hour = item[2]
    minute = date[10:12]
    item[2] = int(item[2])
    #lon = float("{0:.4f}".format(item[4]))
    #lat = float("{0:.4f}".format(item[5]))
    lon = item[4]
    lat = item[5]
    newData.append([item[0],int(year),int(month),int(day),int(hour),int(minute),lon,lat,item[3]])
data = np.array(newData)

In [ ]:
norm = normalizer()
dLength = np.shape(data)[0]

trainSize = int(dLength*0.80)

X_train = norm.fit_transform(data[:trainSize,[2,3,4,5,6,7]])
y_train = data[:trainSize,8]
X_test = norm.fit_transform(data[trainSize:,[2,3,4,5,6,7]])
y_test = data[trainSize:,8]
len(y_train)